<!-- ---
title: How to do time profiling
downloads: true
sidebar: true
tags:
  - time-profiling
  - BasicTimeProfiler
  - Timer
  - HandlersTimeProfiler
--- -->
# How to do time profiling

This example demonstrates how you can get the time breakdown for:
- All epochs during training
- Individual `Events`
- All `Handlers` correspoding to an `Event`
- Individual `Handlers`
- Data loading and Data processing.

In this example, we will be using a ResNet18 model on the MNIST dataset. The base code is the same as used in the Getting Started Guide.

## Basic Setup

In [3]:
import pandas as pd

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision.models import resnet18
from torchvision.transforms import Compose, Normalize, ToTensor

from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss
from ignite.handlers import Timer
from ignite.contrib.handlers import BasicTimeProfiler, HandlersTimeProfiler

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.model = resnet18(num_classes=10)
        self.model.conv1 = nn.Conv2d(1, 64, kernel_size=3, padding=1, bias=False)

    def forward(self, x):
        return self.model(x)


model = Net().to(device)

data_transform = Compose([ToTensor(), Normalize((0.1307,), (0.3081,))])

train_loader = DataLoader(
    MNIST(download=True, root=".", transform=data_transform, train=True),
    batch_size=128,
    shuffle=True,
)

val_loader = DataLoader(
    MNIST(download=True, root=".", transform=data_transform, train=False),
    batch_size=256,
    shuffle=False,
)

optimizer = torch.optim.RMSprop(model.parameters(), lr=0.005)
criterion = nn.CrossEntropyLoss()

We attach two handlers to the `trainer` to print out the metrics ([`Accuracy`](https://pytorch.org/ignite/generated/ignite.metrics.Accuracy.html#accuracy) and [`Loss`](https://pytorch.org/ignite/generated/ignite.metrics.Loss.html#loss)) of the train and validation dataset at the end of every epoch.

In [5]:
trainer = create_supervised_trainer(model, optimizer, criterion, device=device)
evaluator = create_supervised_evaluator(
    model, metrics={"accuracy": Accuracy(), "loss": Loss(criterion)}, device=device
)


@trainer.on(Events.EPOCH_COMPLETED)
def log_training_results(trainer):
    evaluator.run(train_loader)
    metrics = evaluator.state.metrics
    print(
        f"Training Results - Epoch[{trainer.state.epoch}] Avg accuracy: {metrics['accuracy']:.2f} Avg loss: {metrics['loss']:.2f}"
    )


@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_results(trainer):
    evaluator.run(val_loader)
    metrics = evaluator.state.metrics
    print(
        f"Validation Results - Epoch[{trainer.state.epoch}] Avg accuracy: {metrics['accuracy']:.2f} Avg loss: {metrics['loss']:.2f}"
    )

## Using `State` of Events

If we just want to print the time taken for every epoch and the total time for training we can simply use the `trainer`'s [`State`](https://pytorch.org/ignite/generated/ignite.engine.events.State.html#ignite.engine.events.State). We attach two separate handlers fired when an epoch is completed and when the training is completed to log the time returned by `trainer.state.times`.

In [6]:
@trainer.on(Events.EPOCH_COMPLETED)
def log_epoch_time():
    print(
        f"Epoch {trainer.state.epoch}, Time Taken : {trainer.state.times['EPOCH_COMPLETED']}"
    )


@trainer.on(Events.COMPLETED)
def log_total_time():
    print(f"Total Time: {trainer.state.times['COMPLETED']}")

In [7]:
trainer.run(train_loader, max_epochs=2)

Training Results - Epoch[1] Avg accuracy: 0.95 Avg loss: 0.19
Validation Results - Epoch[1] Avg accuracy: 0.95 Avg loss: 0.18
Epoch 1, Time Taken : 30.887829065322876
Training Results - Epoch[2] Avg accuracy: 0.98 Avg loss: 0.05
Validation Results - Epoch[2] Avg accuracy: 0.98 Avg loss: 0.05
Epoch 2, Time Taken : 30.8903865814209
Total Time: 90.81205654144287


State:
	iteration: 938
	epoch: 2
	epoch_length: 469
	max_epochs: 2
	output: 0.04038083925843239
	batch: <class 'list'>
	metrics: <class 'dict'>
	dataloader: <class 'torch.utils.data.dataloader.DataLoader'>
	seed: <class 'NoneType'>
	times: <class 'dict'>

## Event-based profiling using `BasicTimeProfiler`

If we want more information such as the time taken by data processing, data loading and all pre-defined events, we can use [`BasicTimeProfiler()`](https://pytorch.org/ignite/generated/ignite.contrib.handlers.time_profilers.BasicTimeProfiler.html#basictimeprofiler).

In [8]:
# Attach basic profiler
basic_profiler = BasicTimeProfiler()
basic_profiler.attach(trainer)

trainer.run(train_loader, max_epochs=2)

Training Results - Epoch[1] Avg accuracy: 0.99 Avg loss: 0.03
Validation Results - Epoch[1] Avg accuracy: 0.99 Avg loss: 0.04
Epoch 1, Time Taken : 30.75150728225708
Training Results - Epoch[2] Avg accuracy: 0.99 Avg loss: 0.04
Validation Results - Epoch[2] Avg accuracy: 0.99 Avg loss: 0.04
Epoch 2, Time Taken : 30.99610710144043
Total Time: 90.9643816947937


State:
	iteration: 938
	epoch: 2
	epoch_length: 469
	max_epochs: 2
	output: 0.02857031859457493
	batch: <class 'list'>
	metrics: <class 'dict'>
	dataloader: <class 'torch.utils.data.dataloader.DataLoader'>
	seed: <class 'NoneType'>
	times: <class 'dict'>

We can then obtain the results dictionary via [`get_results()`](https://pytorch.org/ignite/generated/ignite.contrib.handlers.time_profilers.BasicTimeProfiler.html#ignite.contrib.handlers.time_profilers.BasicTimeProfiler.get_results) and pass it to [`print_results()`](https://pytorch.org/ignite/generated/ignite.contrib.handlers.time_profilers.BasicTimeProfiler.html#ignite.contrib.handlers.time_profilers.BasicTimeProfiler.print_results) to get a nicely formatted output which contains total, minimum, maximum, mean and the standard deviation of the time taken.

In [9]:
results = basic_profiler.get_results()
basic_profiler.print_results(results)


 ----------------------------------------------------
| Time profiling stats (in seconds):                 |
 ----------------------------------------------------
total  |  min/index  |  max/index  |  mean  |  std

Processing function:
46.29968 | 0.04044/937 | 0.08740/0 | 0.04936 | 0.00196

Dataflow:
15.35097 | 0.01235/936 | 0.02857/124 | 0.01637 | 0.00156

Event handlers:
29.22784

- Events.STARTED: []
0.00001

- Events.EPOCH_STARTED: []
0.00001 | 0.00000/1 | 0.00001/0 | 0.00000 | 0.00000

- Events.ITERATION_STARTED: []
0.00213 | 0.00000/221 | 0.00002/703 | 0.00000 | 0.00000

- Events.ITERATION_COMPLETED: []
0.00317 | 0.00000/54 | 0.00002/30 | 0.00000 | 0.00000

- Events.EPOCH_COMPLETED: ['log_training_results', 'log_validation_results', 'log_epoch_time']
29.21511 | 14.52231/1 | 14.69281/0 | 14.60756 | 0.12056

- Events.COMPLETED: ['log_total_time']
0.00004



"\n ----------------------------------------------------\n| Time profiling stats (in seconds):                 |\n ----------------------------------------------------\ntotal  |  min/index  |  max/index  |  mean  |  std\n\nProcessing function:\n46.29968 | 0.04044/937 | 0.08740/0 | 0.04936 | 0.00196\n\nDataflow:\n15.35097 | 0.01235/936 | 0.02857/124 | 0.01637 | 0.00156\n\nEvent handlers:\n29.22784\n\n- Events.STARTED: []\n0.00001\n\n- Events.EPOCH_STARTED: []\n0.00001 | 0.00000/1 | 0.00001/0 | 0.00000 | 0.00000\n\n- Events.ITERATION_STARTED: []\n0.00213 | 0.00000/221 | 0.00002/703 | 0.00000 | 0.00000\n\n- Events.ITERATION_COMPLETED: []\n0.00317 | 0.00000/54 | 0.00002/30 | 0.00000 | 0.00000\n\n- Events.EPOCH_COMPLETED: ['log_training_results', 'log_validation_results', 'log_epoch_time']\n29.21511 | 14.52231/1 | 14.69281/0 | 14.60756 | 0.12056\n\n- Events.COMPLETED: ['log_total_time']\n0.00004\n"

**Note**: This approach does not get the time taken by an individual handler rather the sum of the time taken by all handlers corresponding to a pre-defined event.

## Handler-based profiling using `HandlersTimeProfiler`

We can overcome the above problem by using [`HandlersTimeProfiler`](https://pytorch.org/ignite/generated/ignite.contrib.handlers.time_profilers.HandlersTimeProfiler.html#handlerstimeprofiler) which gives us only the necessary information. We can also calculate the time taken by handlers attached to [`Custom Events`](https://pytorch.org/ignite/concepts.html#custom-events), which was not previously possible, via this.

In [10]:
# Attach handlers profiler
handlers_profiler = HandlersTimeProfiler()
handlers_profiler.attach(trainer)

In [11]:
trainer.run(train_loader, max_epochs=2)

Training Results - Epoch[1] Avg accuracy: 0.99 Avg loss: 0.03
Validation Results - Epoch[1] Avg accuracy: 0.99 Avg loss: 0.04
Epoch 1, Time Taken : 31.178053617477417
Training Results - Epoch[2] Avg accuracy: 0.99 Avg loss: 0.02
Validation Results - Epoch[2] Avg accuracy: 0.99 Avg loss: 0.04
Epoch 2, Time Taken : 31.31934642791748
Total Time: 91.76212525367737


State:
	iteration: 938
	epoch: 2
	epoch_length: 469
	max_epochs: 2
	output: 0.030506571754813194
	batch: <class 'list'>
	metrics: <class 'dict'>
	dataloader: <class 'torch.utils.data.dataloader.DataLoader'>
	seed: <class 'NoneType'>
	times: <class 'dict'>

We can print the results of the profiler in the same way as above. The output shows total, average and other details of execution time for each handler attached. It also shows the data processing and data loading times.

In [12]:
results = handlers_profiler.get_results()
handlers_profiler.print_results(results)


---------------------------------------  -------------------  --------------  --------------  --------------  --------------  --------------  
Handler                                  Event Name                 Total(s)      Min(s)/IDX      Max(s)/IDX         Mean(s)          Std(s)  
---------------------------------------  -------------------  --------------  --------------  --------------  --------------  --------------  
log_training_results                     EPOCH_COMPLETED              25.236      12.57144/0      12.66456/1          12.618         0.06584  
log_validation_results                   EPOCH_COMPLETED             4.02691       1.97243/0       2.05448/1         2.01346         0.05802  
log_epoch_time                           EPOCH_COMPLETED               7e-05         3e-05/0         4e-05/1           3e-05             0.0  
BasicTimeProfiler._as_first_started      STARTED                     0.00093       0.00093/0       0.00093/0         0.00093            None 

The profiling results obtained by `basic_profiler` and `handler_profiler` can be exported to a CSV file by using the `write_results()` method.

In [13]:
basic_profiler.write_results("./basic_profile.csv")
handlers_profiler.write_results("./handlers_profile.csv")

If we inspect the CSV file of `basic_profiler` we can see the depth of information stored for every iteration.

In [14]:
basic_profile = pd.read_csv("./basic_profile.csv")
basic_profile.head()

,epoch,iteration,processing_stats,dataflow_stats,Event_STARTED,Event_COMPLETED,Event_EPOCH_STARTED,Event_EPOCH_COMPLETED,Event_ITERATION_STARTED,Event_ITERATION_COMPLETED,Event_GET_BATCH_STARTED,Event_GET_BATCH_COMPLETED
0,1.0,1.0,0.087367,0.016016,0.000021,0.000084,0.000004,14.543953,0.000003,0.000008,0.000005,0.000010
1,1.0,2.0,0.087184,0.024107,0.000021,0.000084,0.000004,14.543953,0.000004,0.000007,0.000004,0.000011
2,1.0,3.0,0.060560,0.016312,0.000021,0.000084,0.000004,14.543953,0.000003,0.000006,0.000004,0.000021
3,1.0,4.0,0.052990,0.016108,0.000021,0.000084,0.000004,14.543953,0.000003,0.000006,0.000004,0.000008
4,1.0,5.0,0.053011,0.017612,0.000021,0.000084,0.000004,14.543953,0.000003,0.000006,0.000006,0.000007


The `handlers_profile` CSV stores the details for whenever a handler was evoked which corresponds to the number of rows. 

In [15]:
handlers_profile = pd.read_csv("./handlers_profile.csv")
handlers_profile.head()

,#,processing_stats,dataflow_stats,log_training_results (EPOCH_COMPLETED),log_validation_results (EPOCH_COMPLETED),log_epoch_time (EPOCH_COMPLETED),BasicTimeProfiler._as_first_started (STARTED),log_total_time (COMPLETED)
0,1.0,0.087421,0.020330,12.571441,1.97243,0.000032,0.00093,0.00003
1,2.0,0.087228,0.015991,12.664556,2.05448,0.000037,0.00000,0.00000
2,3.0,0.060598,0.024079,0.000000,0.00000,0.000000,0.00000,0.00000
3,4.0,0.053027,0.016274,0.000000,0.00000,0.000000,0.00000,0.00000
4,5.0,0.053047,0.016084,0.000000,0.00000,0.000000,0.00000,0.00000


## Custom profiling using Timer

At the lowest level of abstraction, we provide [`Timer()`](https://pytorch.org/ignite/generated/ignite.handlers.timing.Timer.html#timer) to calculate the time between any set of events. See its docstring for details.